# 1.1 Serving a Public Model (GPT-2)
**Protecting Individual's data privacy**

## Dependencies

Make you sure you have [pytorch-transformers](https://github.com/huggingface/pytorch-transformers) installed. PyTorch-Transformers can be installed by pip as follows:

In [1]:
# !pip install pytorch-transformers

In [2]:
import torch as th
import grid as gr
from grid import syft as sy

# GPT-2 model
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

In [3]:
hook = sy.TorchHook(th)

## Connect to a GridWorker

In [4]:
app_company = gr.WebsocketGridClient(hook, "http://localhost:3000/", id="app_company")
app_company.connect()

## Model and Data Setup

In [6]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2',torchscript=True)

In [7]:
# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = th.tensor([indexed_tokens])

IMPORTANT: if you trace the model with a tensor of size `N` the trace model only expects inputs of size `N`.

In [8]:
traced_model = th.jit.trace(model, (tokens_tensor,))

/home/marianne/PySyft/syft/generic/frameworks/hook/hook.py:342: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  response = method(*args, **kwargs)
/home/marianne/PySyft/syft/generic/frameworks/hook/hook.py:342: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  response = method(*args, **kwargs)


## Hosting a Public Model

The important part is that we set `allow_download=True` so a user can download a copy of the model to run locally.

In [9]:
# Host GPT-2 on Bob worker
app_company.serve_model(traced_model, model_id="GPT-2-public", allow_download=True, allow_remote_inference=False)

True